In [1]:
# python manage.py shell_plus --notebook
import os
import pandas as pd
import django
from django.core.files.uploadedfile import UploadedFile
from wfrp.models import (Campaign, Item, Talent, Spell, AdvancedSkill,
                         PlayableCharacter, Armour, Weapon)
from users.models import User
from wfrp.scripts.default_content import addDefaultCharacter

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
EXAMPLE_MEDIA_ROOT = '../example_media/'
DEFAULT_CHARACTER_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_characters.xlsx')
ARMOUR_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_armour.xlsx')
ITEM_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_items.xlsx')
WEAPON_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_weapons.xlsx')
SPELL_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_spells.xlsx')
TALENT_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_talents.xlsx')
ADVANCEDSKILL_FILE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_advanced_skills.xlsx')
DEFAULT_CHARACTERS = ['GUNNAR', 'MOLRELLA',
                      'FERDINAND', 'AMRIS', 'ELSE', 'BLANK']

### Purge current instances

In [3]:
PlayableCharacter.objects.all().delete()
Item.objects.all().delete()
Armour.objects.all().delete()
Talent.objects.all().delete()
Spell.objects.all().delete()
AdvancedSkill.objects.all().delete()
Weapon.objects.all().delete()
# User.objects.all().delete()
User.objects.all().exclude(username='admin').delete()
Creature.objects.all().delete()
CreatureTrait.objects.all().delete()

(0, {})

### Create test users

In [4]:
users = [{'name': 'ricardo'},
         {'name': 'danna'},
         {'name': 'omar'},
         {'name': 'caceres'},
         {'name': 'aleja'},
         {'name': 'elliot'},
         {'name': 'felipe'},
         {'name': 'velez'}]

for user in users:
    user_instance = User.objects.create_user(username=user['name'], 
                                             password=f'{user["name"]}_changeme')
    user_instance.save()

admin = User.objects.filter(username='admin')[0]
# admin = User.objects.create_user('admin', password='123')
# admin.is_superuser = True
# admin.is_staff = True
# admin.save()

### Create default campaign

In [5]:
campaign_master = User.objects.get(username='velez')
campaign_name = 'Haciendo la ronda'
campaign_description = '''
Los Personajes, que paseaban por el mercado de Ubersreik, se ven metidos en un tumulto y, más tarde, se les acusa de instigarlo. Misteriosamente, una abogada local interviene y convence al juez responsable del caso de que permita a los Personajes que paguen su ‘deuda’ trabajando como miembros de la Guardia.

Pronto, el grupo se ve patrullando las calles de Ubersreik con Rudi Klumpenklug, un sargento de la guardia absolutamente corrupto. Los guardias temporales se ven expuestos a diversos delitos, ninguno de los cuales Klumpenklug muestra interés alguno por perseguir, dejando que los Personajes resuelvan, ignoren o incluso exploten cada situación como prefieran.

Por último, Ilse Fassenwütend se pone en contacto con el grupo. Fassenwütend es una guarda de caminos, que afirma ser capaz de hacer que la sentencia de los Personajes sea conmutada si le ayudan a escoltar a un criminal al lugar donde debe ser ejecutado.

Sin embargo, no será tarea fácil. El grupo tendrá ante sí una noche horrorosa, llevando a un hombre aterrorizado a través de unas calles extrañamente alteradas mientras Morrslieb, la luna del Caos, está en cuarto creciente (y por lo tanto brilla en el cielo) y sectarios mutantes los atacan por todos lados ¿Conseguirán los Personajes proteger al hombre, que afirma ser inocente, para luego llevarlo hasta el cadalso? ¿O elegirán un camino distinto?
'''

if len(Campaign.objects.filter(name=campaign_name)) == 0:
  campaign = Campaign(master=campaign_master, 
                      name=campaign_name,
                      description=campaign_description.strip())
  campaign.save()
  campaign.cover_image.save(f'ubersreik.jpg', UploadedFile(file=open(f'../example_media/ubersreik.jpg', 'rb'), content_type='image/png'))
else:
  campaign = Campaign.objects.get(name=campaign_name)

### Add default characters

In [6]:
user = User.objects.filter(username='ricardo')[0]
campaign = Campaign.objects.all()[0]
addDefaultCharacter(user=user, campaign=campaign, character_name='GUNNAR')
character_instance = PlayableCharacter.objects.filter(name='Gunnar Hrolfsson')[0]

### Add default creatures

In [7]:
from wfrp.models import Creature
DEFAULT_CREATURE_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_creatures.xlsx')
creature_data = pd.read_excel(DEFAULT_CREATURE_PATH)
creature_data = creature_data.to_dict('records')
for item in creature_data:
    pic_path = os.path.join(EXAMPLE_MEDIA_ROOT, item['creature_picture'])
    # item['user'] = admin
    # item['campaign'] = campaign
    item['creature_picture'] = UploadedFile(file=open(pic_path, 'rb'),
                                                     content_type='image/png')

Creature.objects.bulk_create([Creature(**item) for item in creature_data])

[<Creature: Gor_bdb82f2c-3687-4a8e-94b4-f756302ef576>,
 <Creature: Ungor_ac7b21d6-a143-46f8-ba08-47dfd9859359>]

In [8]:
from wfrp.models import CreatureTrait
DEFAULT_CREATURETRAIT_PATH = os.path.join(EXAMPLE_MEDIA_ROOT, 'default_creature_traits.xlsx')
creature_traits = pd.read_excel(DEFAULT_CREATURETRAIT_PATH)
creature_traits = creature_traits.to_dict('records')
for item in creature_traits:
    creature_name = item['creature_name']
    creature = Creature.objects.filter(name=creature_name)[0]
    _ = item.pop('creature_name', '')
    # item['user'] = admin
    # item['campaign'] = campaign
    trait = CreatureTrait(**item)
    trait.save()
    creature.traits.add(trait)